## Introduction to TensorBoard 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/easy-tensorflow/easy-tensorflow/blob/master/1_TensorFlow_Basics/Tutorials/3_Introduction_to_Tensorboard.ipynb)

In the first post, we discussed the advantages of TensorFlow. They were mainly flexibility and visualization. Imagine if you can visualize whats happening in the code (in this case code represents the computational graph that we create for a model), it would be so convenient to deeply understand and observe the inner workings of the graph. Not just that, it also helps in fixing things that are not working the way they should. TensorFlow provides a way to do just that using TensorBoard!

__TensorBoard__ is a visualization software that comes with any standard TensorFlow installation. In Google’s words: “The computations you'll use TensorFlow for many things (like training a massive deep neural network) and they can be complex and confusing. To make it easier to understand, debug, and optimize TensorFlow programs, we've included a suite of visualization tools called TensorBoard.”

TensorFlow programs can range from a very simple to super complex problems (using thousands of computations), and they all have two basic components, Operations, and Tensors. As explained in the previous tutorials, the idea is that you create a model that consists of a set of operations, feed the data into the model and the tensors will flow between the operations until you get an output tensor i.e., your result. TensorBoard provides us with a suite of web applications that help us to inspect and understand the TensorFlow runs and graphs. Currently, it provides five types of visualizations: scalars, images, audio, histograms, and graphs.

When fully configured, TensorBoard window will look something like:

<img src="files/files/3_1.png" width="500" height="1000">


___Fig. 1. ___ TensorBoard appearance


TensorBoard was created as a way to help us understand the flow of tensors in your model so that we can debug and optimize it. It is generally used for two main purposes:

__1. Visualizing the Graph__

__2. Writing Summaries to Visualize Learning__

We'll cover the main usages of TensorBoard in this tutorial. Learning to use TensorBoard early and often will make working with TensorFlow much more enjoyable and productive.

## 1. Visualizing the Graph

While powerful, TensorFlow computation graphs can become extremely complicated. Visualizing the graph can help us understand and debug it. Here's an example of the visualization at work from TensorFlow website.

<img src="files/files/3_2.gif" width="500" height="1000" >
 
___Fig. 2. ___ Visualization of a TensorFlow graph (Source: TensorFlow website)

To make our TensorFlow code __TensorBoard-activated__, we need to add some lines of code. This will export the TensorFlow operations into a file, called __event file__ (or event log file). TensorBoard is able to read this file and give some insights of the model graph and its performance.

Now let's write a simple TensorFlow code and visualize its computational graph with TensorBoard.

### Example 1:
Let's create two constants and add them together. As we say in the previous tutorial, we can create a `tf.function` to create our graph:


In [1]:
import tensorflow as tf

# create tf.function
@tf.function
def simple_add(a, b):
    # create the graph
    c = tf.add(a, b, name="addition")
    return c

a = tf.constant(2, name="a")
b = tf.constant(3, name="b")

# execute the function
c = simple_add(a, b)
print(c)

tf.Tensor(5, shape=(), dtype=int32)


To visualize the graph with TensorBoard, we need to write log. To write event files, we first need to create a __writer__ for those logs, using the following template code:

```python
writer = tf.summary.FileWriter([logdir], [graph])
```

where

__[logdir]__ is the folder where we want to store those log files. We can also choose [logdir] to be something meaningful such as './graphs'.

__[graph]__ is the graph of the function we're working on. We can get the graph by using the following code:
```python
tf_function.get_concrete_function(*args).graph
```
For example, in our case, we can get the graph by:
```python
simple_add.get_concrete_function(a, b).graph
```

Let's add the writer to the first example and visualize the graph.


In [2]:
# creating the writer out of the session
logdir = './graphs'
writer = tf.summary.create_file_writer(logdir)

# Write the traced graph to TensorBoard
with writer.as_default():
    tf.summary.graph(simple_add.get_concrete_function(a, b).graph)
    writer.flush()

print(f"TensorBoard logs saved to: {logdir}")

TensorBoard logs saved to: ./graphs


This code will create  a directory inside your current directory (beside your Python code file) which contains the __event file__.

<img src="files/files/3_3.png" width="300" height="600" >

___Fig. 3. ___ Created directory which contains the event file


Next, to visualize the graph, we need to go to Terminal and make sure that the present working directory is the same as where we ran our Python code. For example, here we can switch to the directory using the commands:

```bash
cd ~/Desktop/tensorboard
tensorboard --logdir="./graphs" --host localhost --port 6006
```

Replace './graphs' with the name of the directory in case you choose to name it something else. This will generate a link on the command line. Control click (ctrl+left) the link to open the TensorBoard window, TensorBoard uses the web browser to show us the visualizations (or simply copy it into your browser or just open your browser and go to http://localhost:6006/). The link will direct us to the TensorBoard page, it should look similar to:

<img src="files/files/3_4.png" width="500" height="1000" >

___Fig. 4. ___ TensorBoard page visualizing the graph generated in Example 1


If you are running the code in a Jupyter notebook, you can use the following code to open TensorBoard in a new window:

In [3]:
%load_ext tensorboard
%tensorboard --logdir="./graphs" --host localhost --port 6006
print("Click on the link to open TensorBoard in a new window: http://localhost:6006/")

Click on the link to open TensorBoard in a new window: http://localhost:6006/


__*Note:__ If we run our code several times with the same [logdir], multiple event files will be generated in our [logdir]. TF will only show the latest version of the graph and display the warning of multiple event files. The warning can be removed by deleting the event files that we no longer need or else we can save them in a different [logdir] folder.

## 2. Writing Summaries to Visualize Learning

So far we only focused on how to visualize the graph in TensorBoard. Remember the other types of visualizations mentioned in the earlier part of the post that TensorBoard provides (scalars, images and histograms). In this part, we are going to use a special operation called __summary__ to visualize the model parameters (like weights and biases of a neural network), metrics (like loss or accuracy value), and images (like input images to a network). 

__Summary__ is a special operation TensorBoard that takes in a regular tensor and outputs the summarized data to your disk (i.e. in the event file). There are several types of summaries in TensorFlow ([read more](https://www.tensorflow.org/api_docs/python/tf/summary)), but we will focus on the following three types in this post:

__1. tf.summary.scalar:__ used to write a single scalar-valued tensor (like a classification loss or accuracy value)

__2. tf.summary.histogram:__ used to plot histogram of all the values of a non-scalar tensor (can be used to visualize weight or bias matrices of a neural network)

__3. tf.summary.image:__ used to plot images (like input images of a network, or generated output images of an autoencoder or a GAN)

In the following sections, let's go through each of the above mentioned summary types in more detail.


### 2.1. tf.summary.scalar:
It's for writing the values of a scalar tensor that changes over time or iterations. In the case of neural networks (say a simple network for classification task), it's usually used to monitor the changes of loss function or classification accuracy.

Let's run a simple example to understand the point.

### Example 2:
Randomly pick 100 values from a standard Normal distribution, _N(0, 1)_, and plot them one after the other.

One way to do so is to create a scalar variable and write its value to the __summary__ using __tf.summary.scalar__. We can then write the summary to the disk using a __tf.summary.FileWriter__. The code is as follows:


In [4]:
import tensorflow as tf

# create a summary writer to write the summaries to disk
summary_write = tf.summary.create_file_writer('./summaries_scalar')

with summary_write.as_default():
    for step in range(100):
        # create a random scalar variable
        random_scalar = tf.random.normal([], mean=0, stddev=1)
        
        # write the scalar summary
        tf.summary.scalar('Random_scalar', random_scalar, step=step)
        
print('Done writing the summaries')

Done writing the summaries


Let's pull up TensorBoard and checkout the result. Like before, you need to open terminal and type:

```bash
tensorboard --logdir="./summaries_scalar" --host localhost --port 6007
```

__*Note:__ Since we used port 6006 in the previous example, we need to use a different port (6007) for this example.

If you are running the code in a Jupyter notebook, you can use the following code to open TensorBoard in a new window:

In [5]:
# Reload the extension if it was already loaded from the previous example
%reload_ext tensorboard
%tensorboard --logdir="./summaries_scalar" --host localhost --port 6007
print("Click on the link to open TensorBoard in a new window: http://localhost:6007/")

Click on the link to open TensorBoard in a new window: http://localhost:6007/


Here _"./summaries_scalar"_ is the name of the directory we saved the event file to. In TensorBoard, we find a new tab named __"scalars"__. The whole window looks like:

<img src="files/files/3_6.png" width="500" height="1000" >

___Fig. 6. ___ TensorBoard page visualizing the written scalar summary.

In the figure, the x-axis and y-axis shows the 100 steps and the corresponding values (random values from a standard normal distribution) of the variable respectively. 

### 2.2. tf.summary.histogram:

Histogram comes in handy if we want to observe the change of a value over time or iterations. It's used for plotting the histogram of the values of a non-scalar tensor. This provides us a view of how the histogram (and the distribution) of the tensor values change over time or iterations. In the case of neural networks, it's commonly used to monitor the changes of weight and biase distributions. It's very useful in detecting irregular behavior of the network parameters (for example, when our weights explode or shrink abnormally). 

### Example 3:
We will create a scalar and matrix of size 30x40, whose entries come from a standard normal distribution. Initialize the variables 100 times and plot their distribution over time.

In [6]:
import tensorflow as tf

# create a summary writer to write the summaries to disk
summary_write = tf.summary.create_file_writer('./summaries_histogram')

with summary_write.as_default():
    for step in range(100):
        
        # create a random matrix variable
        random_matrix = tf.random.normal([30, 40], mean=0, stddev=1)
        
        # write the histogram summary
        tf.summary.histogram('Random_matrix', random_matrix, step=step)
        
print('Done writing the summary histograms')        

Done writing the summary histograms


Similarly, open TensorBoard and checkout the result. Like before, you need to open terminal and type:
```bash
tensorboard --logdir="./summaries_histogram" --host localhost --port 6008
```
Or if you are running the code in a Jupyter notebook, you can use the following code to open TensorBoard in a new window:

In [7]:
%reload_ext tensorboard
%tensorboard --logdir="./summaries_histogram" --host localhost --port 6008
print("Click on the link to open TensorBoard in a new window: http://localhost:6008/")

Click on the link to open TensorBoard in a new window: http://localhost:6008/


In TensorBoard, two new tabs are added to the top menu: "Distributions" and "Histograms". The results will be as follows:

<img src="files/files/3_7.png">

___Fig. 7. ___ (a) scalar summary, (b) distribution and (c) histogram of the values of the 2D-tensor over 100 steps


In the figure, the "Distributions" tab contains a plot that shows the distribution of the values of the tensor (y-axis) through steps (x-axis). You might ask what are the light and dark colors? 

The answer is that each line on the chart represents a percentile in the distribution over the data. For example, the bottom line (the very light one) shows how the minimum value has changed over time, and the line in the middle shows how the median has changed. Reading from top to bottom, the lines have the following meaning: [maximum, 93%, 84%, 69%, 50%, 31%, 16%, 7%, minimum]

These percentiles can also be viewed as standard deviation boundaries on a normal distribution: [maximum, μ+1.5σ, μ+σ, μ+0.5σ, μ, μ-0.5σ, μ-σ, μ-1.5σ, minimum] so that the colored regions, read from inside to outside, have widths [σ, 2σ, 3σ] respectively.

Similarly, in the histogram panel, each chart shows temporal "slices" of data, where each slice is a histogram of the tensor at a given step. It's organized with the oldest timestep in the back, and the most recent timestep in front.

You can easily monitor the values on the histograms at any step. Just move your cursor on the plot and see the x-y values on the histograms (Fig8 (a)). You can also change the Histogram Mode from "offset" to "overlay" (see Fig. 8- (b)) to see the histograms overlaid with one another.


<img src="files/files/3_8.png">

___Fig. 8. ___ (a) monitor values on the histograms, (b) overlayed histograms 

### 2.3. tf.summary.image:
As the name implies, this type of summary is used for writing and visualizing tensors as images. In the case of neural networks, this is usually used for tracking the images that are either fed to the network (say in each batch) or the images generated in the output (such as the reconstructed images in an autoencoder; or the fake images made by the generator model of a Generative Adverserial Network). However, in general, this can be used for plotting any tensor. For example, we can visualize a weight matrix of size 30x40 as an image of 30x40 pixels.

An image summary can be created using:

```python
tf.summary.image(name, data, step=None, max_outputs=3)
```

Where `name` is the name for the generated node (i.e. operation), `data` is the desired tensor to be written as an image summary (we will talk about its shape shortly), `step` is the step number for this summary and `max_outputs` is the maximum number images to be emitted per step.

The `data` that we feed to tf.summary.image must be a 4-D tensor of shape `[batch_size, height, width, channels]` where batch_size is the number of images in the batch, the height and width determine the size of the image and finally, the channels are:
1: for Grayscale images.
3: for RGB (i.e. color) images.
4: for RGBA images (where A stands for alpha; see [RGBA](https://en.wikipedia.org/wiki/RGBA_color_space)).

Let's look at a very simple example to get the underlying idea.

### Example 4:
Let's create two random batches of grayscale and color images and plot them as images in TensorBoard.

In [8]:
import tensorflow as tf

# create a summary writer to write the summaries to disk
summary_write = tf.summary.create_file_writer('./summaries_image')

with summary_write.as_default():
    for step in range(100):
        
        # create a random grayscale image
        random_batch_gray = tf.random.normal([3, 10, 10, 1])
        
        # create a random color image
        random_batch_color = tf.random.normal([5, 10, 10, 3])
        
        # write the image summaries
        tf.summary.image('grayscale_images', random_batch_gray, step=step)
        tf.summary.image('color_images', random_batch_color, step=step, max_outputs=5)
        
print('Done writing the summary images')        

Done writing the summary images


Now open TensorBoard like before and switch to __IMAGES__ tab.

In [9]:
%reload_ext tensorboard
%tensorboard --logdir="./summaries_image" --host localhost --port 6009
print("Click on the link to open TensorBoard in a new window: http://localhost:6009/")

Click on the link to open TensorBoard in a new window: http://localhost:6009/


The images should be something similar to:
<img src="files/files/3_9.png">

___Fig. 9. ___ generated images in TensorBoard

We can similarly add any other image of any size to our summaries and plot them in TensorBoard. We'll see more of such summaries in our next tutorials. You can create a Linear Classifier and use TensorBoard using our [Linear Classifier with TensorBoard](https://github.com/easy-tensorflow/easy-tensorflow/blob/master/2_Linear_Classifier/Tutorials/1_Linear_Classifier.ipynb) post.

### Useful Tips on running TensorBoard in Jupyter Notebook
If you are running the code in a Jupyter notebook, you can list the TensorBoard instances running in the background using the following code:

In [10]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6007: logdir ./summaries_scalar (started 0:00:07 ago; pid 18096)
  - port 6006: logdir ./graphs (started 0:00:09 ago; pid 18576)
  - port 6008: logdir ./summaries_histogram (started 0:00:04 ago; pid 19220)
  - port 6009: logdir ./summaries_image (started 0:00:01 ago; pid 6840)


You can also display the TensorBoard in the notebook itself using the following code:

In [11]:
notebook.display(port=6006, height=1000)

Selecting TensorBoard with logdir ./graphs (started 0:00:09 ago; port 6006, pid 18576).


You can delete the TensorBoard instance by deleting the content of the following directory:
```bash
# in Windows
C:\Users\<UserName>\AppData\Local\Temp\.tensorboard-info

# in Linux
/tmp/.tensorboard-info

# in Mac
/private/var/folders/<random>/T/.tensorboard-info
```

I hope this post has helped you to understand the basic key features of __TensorBoard__. Thank you so much for reading! If you have any questions, feel free to leave a comment in our [webpage](http://www.easy-tensorflow.com/basics/introduction-to-tensorboard). You can also send us feedback through the [__contacts__](http://www.easy-tensorflow.com/contacts) page.